In [9]:
from classiq import *
import numpy as np
import math


In [10]:
# Assuming these won't change
x0 = np.array([1, 1])
M = np.array([[0, 1], [-1, 0]])

# Free to change variables
t = 1
K = 10 # Accuracy of solution - subject to change
T = math.ceil(math.log2(K+1))


In [11]:

G = [0] * (2**T)
G[0] = 1
print(G)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:

@qfunc
def encode_stage(ancilla_reg_1: QBit, ancilla_reg_2: QNum, work_reg: QBit):

   def C(m):
      return 2 ** ((m + 1) / 2) * t ** m / math.factorial(m)

   state = []

   for i in range(2 ** T):
      if len(state) < K + 1:
         state.append(math.sqrt(C(i)))

      else:
         state.append(0)

   state = np.array(state) / np.linalg.norm(state)
      
   Z(ancilla_reg_1)
   inplace_prepare_amplitudes(amplitudes=state.tolist(), bound=0.001, target=ancilla_reg_2)
   H(work_reg)



   
@qfunc
def create_entanglement(ancilla_reg_2: QNum, work_reg: QBit):
   repeat(
      count=K + 1,
      iteration=lambda i: if_(
         condition=(i % 2 != 0),
         then=lambda: control(
               ctrl=(ancilla_reg_2 == i), stmt_block=lambda: Y(work_reg)
            ),
         ),
      )

@qfunc
def decode_stage(ancilla_reg_1: QBit, ancilla_reg_2: QNum, work_reg: QBit):

   state = [0] * (2 ** T)
   state[0] = 1
   
   Z(ancilla_reg_1)
   inplace_prepare_amplitudes(amplitudes=state, bound=0.0, target=ancilla_reg_2)
   

In [21]:
@qfunc
def main(ancilla_reg_1: Output[QBit], ancilla_reg_2: Output[QNum], work_reg: Output[QBit]):
   allocate(CInt("1"), ancilla_reg_1)
   allocate(T, ancilla_reg_2)
   allocate(1, work_reg) # 2x2 matrix, will change for higher order ODEs if we get to it

   encode_stage(ancilla_reg_1, ancilla_reg_2, work_reg)
   create_entanglement(ancilla_reg_2, work_reg)
   decode_stage(ancilla_reg_1, ancilla_reg_2, work_reg)

In [22]:
# View circuit on Classiq IDE
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)